# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

geoapify_key

'81164b6c93b84c55923e6da0af83c31b'

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
# C:/Users/jimco/Documents/python-api-challenge/
# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,wainwright,52.8390,-110.8572,-4.10,92,12,3.46,CA,1730646572
1,1,bredasdorp,-34.5322,20.0403,19.43,54,100,11.76,ZA,1730646573
2,2,inhambane,-23.8650,35.3833,26.99,74,0,6.69,MZ,1730646574
3,3,port-aux-francais,-49.3500,70.2167,5.34,83,20,9.68,TF,1730646575
4,4,enewetak,11.3474,162.3373,28.14,78,18,1.83,MH,1730646577


In [3]:
# has_negative = (city_data_df['Humidity'] < 0).any()
# if has_negative:
#     print("There are negative numbers in the 'Humidity' column.")
# else:
#     print("There are no negative numbers in the 'Humidity' column.")

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display
# city_data_df["Humidity_Percentage"] = city_data_df['Humidity'] / 100

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
City_Ideal_Weather_df = city_data_df[
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0)
]
# Drop any rows with null values
City_Ideal_Weather_df =City_Ideal_Weather_df.dropna(how='all')

# Display sample data
City_Ideal_Weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
60,60,saint-pierre,-21.3393,55.4781,24.82,73,0,1.54,RE,1730646646
74,74,al wafrah,28.6392,47.9306,26.18,49,0,3.93,KW,1730646662
132,132,chakulia,22.4833,86.7167,24.05,60,0,1.66,IN,1730646731
154,154,tamanrasset,22.7850,5.5228,25.95,17,0,3.60,DZ,1730646759
193,193,camana,-16.6228,-72.7111,21.27,66,0,4.43,PE,1730646805
308,308,khash,28.2211,61.2158,23.48,17,0,3.33,IR,1730646942
321,321,myitkyina,25.3833,97.4000,22.50,82,0,0.74,MM,1730646958
372,372,ujjain,23.1833,75.7667,25.12,36,0,1.39,IN,1730647020
460,460,itoman,26.1247,127.6694,25.01,78,0,3.60,JP,1730647128
469,469,toliara,-23.3500,43.6667,26.64,75,0,3.99,MG,1730647140


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
Data_to_keep =['City','Country', 'Lat', 'Lng', 'Humidity']
hotel_df=City_Ideal_Weather_df[Data_to_keep].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.loc[:, 'Hotel Name'] = ' ' 

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
60,saint-pierre,RE,-21.3393,55.4781,73,
74,al wafrah,KW,28.6392,47.9306,49,
132,chakulia,IN,22.4833,86.7167,60,
154,tamanrasset,DZ,22.7850,5.5228,17,
193,camana,PE,-16.6228,-72.7111,66,
308,khash,IR,28.2211,61.2158,17,
321,myitkyina,MM,25.3833,97.4000,82,
372,ujjain,IN,23.1833,75.7667,36,
460,itoman,JP,26.1247,127.6694,78,
469,toliara,MG,-23.3500,43.6667,75,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
categories ="accommodation.hotel"
filters ={}
bias ={}

limit = 20


params = {
    "categories":categories,
    #"conditions":conditions,
    "limit":limit,
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key    
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat = row["Lat"]
    Lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    #print(requests.get(base_url, params=params))

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
al wafrah - nearest hotel: No hotel found
chakulia - nearest hotel: No hotel found
tamanrasset - nearest hotel: فندق الأمان
camana - nearest hotel: Caja Arequipa
khash - nearest hotel: No hotel found
myitkyina - nearest hotel: New Light Hotel
ujjain - nearest hotel: Meghdoot luxury hotel and resort
itoman - nearest hotel: No hotel found
toliara - nearest hotel: Ambary
san jose de jachal - nearest hotel: San Martin


,City,Country,Lat,Lng,Humidity,Hotel Name
60,saint-pierre,RE,-21.3393,55.4781,73,Tropic Hotel
74,al wafrah,KW,28.6392,47.9306,49,No hotel found
132,chakulia,IN,22.4833,86.7167,60,No hotel found
154,tamanrasset,DZ,22.7850,5.5228,17,فندق الأمان
193,camana,PE,-16.6228,-72.7111,66,Caja Arequipa
308,khash,IR,28.2211,61.2158,17,No hotel found
321,myitkyina,MM,25.3833,97.4000,82,New Light Hotel
372,ujjain,IN,23.1833,75.7667,36,Meghdoot luxury hotel and resort
460,itoman,JP,26.1247,127.6694,78,No hotel found
469,toliara,MG,-23.3500,43.6667,75,Ambary


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 500,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)